In [24]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from random import shuffle
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from sklearn.preprocessing import OneHotEncoder,normalize, StandardScaler


In [25]:
fer_path = '../Data/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013/fer2013.csv'
data = pd.read_csv(fer_path)

In [26]:
np.unique(data['Usage'], return_counts=True)

(array(['PrivateTest', 'PublicTest', 'Training'], dtype=object),
 array([ 3589,  3589, 28709]))

In [27]:
training_pixels = data[data['Usage'] == 'Training']['pixels'].tolist()
width, height = 48, 48
image_size = (width, height)
X_train = []
for pixel_sequence in training_pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    #     face = cv2.resize(face.astype('uint8'), image_size)
    X_train.append(face.astype('float32'))
    
X_train = np.asarray(X_train)
# X_train = np.expand_dims(X_train, -1)
X_train = X_train.reshape(X_train.shape[0], -1)
# y_train = pd.get_dummies(data[data['Usage'] == 'Training']['emotion']).values
y_train = data[data['Usage'] == 'Training']['emotion'].values

In [28]:
training_pixels = data[data['Usage'] == 'PublicTest']['pixels'].tolist()
width, height = 48, 48
image_size = (width, height)
X_test = []
for pixel_sequence in training_pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    #     face = cv2.resize(face.astype('uint8'), image_size)
    X_test.append(face.astype('float32'))
X_test = np.asarray(X_test)

X_test = np.expand_dims(X_test, -1)
y_test = pd.get_dummies(data[data['Usage'] == 'PublicTest']['emotion']).values



In [31]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(X_train[0].shape)

(28709, 2304)
(3589, 48, 48, 1)
(28709,)
(3589, 7)
(2304,)


Faces shape type is (#records, image_width, image_height, 1)

In [32]:
IMG_SIZE = 48
model = Sequential()
model.add(Conv2D(64, kernel_size = (3, 3), activation='relu', input_shape=(2304,)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
# model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(7, activation = 'softmax'))

ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=2

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=len(X_train)//20, epochs=20, verbose=1)

Epoch 1/20
28709/28709 [==============================] - 294s - loss: 1.9508 - acc: 0.3145   
Epoch 2/20
28709/28709 [==============================] - 305s - loss: 1.6046 - acc: 0.4030   
Epoch 3/20
28709/28709 [==============================] - 300s - loss: 1.4849 - acc: 0.4463   
Epoch 4/20
28709/28709 [==============================] - 293s - loss: 1.4170 - acc: 0.4730   
Epoch 5/20
28709/28709 [==============================] - 318s - loss: 1.3557 - acc: 0.4882   
Epoch 6/20
28709/28709 [==============================] - 309s - loss: 1.2932 - acc: 0.5134   
Epoch 7/20
28709/28709 [==============================] - 316s - loss: 1.1974 - acc: 0.5487   
Epoch 8/20
28709/28709 [==============================] - 320s - loss: 1.1770 - acc: 0.5587   
Epoch 9/20
28709/28709 [==============================] - 320s - loss: 1.1492 - acc: 0.5730   
Epoch 10/20
28709/28709 [==============================] - 323s - loss: 1.1190 - acc: 0.5867   
Epoch 11/20
28709/28709 [========================

In [17]:
model.save('keras_on_fer')

In [16]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Testing loss: {}, acc: {}\n'.format(loss, acc))

Testing loss: 1.4809648709464585, acc: 0.5293953747894147



In [18]:
def detectFaceOpenCVDnn(net, frame):
    result = []
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], False, False)
    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    conf_threshold = 0.7
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            if x1 > frameWidth or x1 < 0 or x2 > frameWidth or x2 < 0 or y1 < 0 or y1 > frameHeight or y2 < 0 or y2 > frameHeight:
                continue
            else:
                grayOpenDnn = gray = cv2.cvtColor(frameOpencvDnn, cv2.COLOR_BGR2GRAY)
                croppedOpenDnn = cv2.resize(gray[y1:y2,x1:x2], (48,48)) 
                result.append(croppedOpenDnn)
    return result, bboxes

In [19]:
def face_reduction(image):
    # OpenCV DNN supports 2 networks.
    # 1. FP16 version of the original caffe implementation ( 5.4 MB )
    # 2. 8 bit Quantized version using Tensorflow h( 2.7 MB )
#     print("printing image")
#     print(image)
    DNN = "TF"
    if DNN == "CAFFE":
        modelFile = "models/res10_300x300_ssd_iter_140000_fp16.caffemodel"
        configFile = "models/deploy.prototxt"
        net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    else:
        modelFile = "models/opencv_face_detector_uint8.pb"
        configFile = "models/opencv_face_detector.pbtxt"
        net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

    conf_threshold = 0.7
    outOpencvDnn, bboxes = detectFaceOpenCVDnn(net,image)
    return outOpencvDnn, bboxes

In [20]:
def display_emotions(videoFaces, frame, bboxes):
    outputFrame = frame.copy()
    predictions = model.predict(videoFaces)
    for i in range(len(bboxes)):
        emotion = y_translation[np.argmax(predictions[i])]
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(outputFrame, emotion, (bboxes[i][0] -1 ,bboxes[i][1] - 1), font, 1, (0,255,0), 2, cv2.LINE_AA)
        cv2.rectangle(outputFrame, (bboxes[i][0], bboxes[i][1]), (bboxes[i][2], bboxes[i][3]), (0, 255, 0), int(round(outputFrame.shape[0]/150)), 8)
    return outputFrame

In [23]:
def get_webcam_feed():
    cap = cv2.VideoCapture(0)
    hasFrame, frame = cap.read()
    #TODO : Handle cases for multiple people
    frame_count = 0
    tt_opencvDnn = 0
    while(1):
        try:
            hasFrame, frame = cap.read()
            if not hasFrame:
                break
            frame_count += 1
            outputFrame = frame
            #Gives a list of gray-scale images in webcam feed
            videoFaces, bboxes = face_reduction(frame)
            videoFaces = np.array(videoFaces)
            if videoFaces.shape[0] != 0:
                print(videoFaces.shape)
                videoFaces = videoFaces.reshape((videoFaces.shape[0],videoFaces.shape[1],videoFaces.shape[2],1))
                outputFrame = display_emotions(videoFaces, frame, bboxes)
            cv2.imshow("frame", outputFrame)
            k = cv2.waitKey(10)
            if k == 27:
                break
        except Exception as e:
            print("Exception is ")
            print(e)
            break
    cv2.destroyAllWindows()

In [30]:
y_translation = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Neutral", 5:"Surprise", 6:"Sad"}

In [32]:
get_webcam_feed()

(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 